In [43]:
!pip install openai
!pip install python-dotenv

# From Zero to CoT

This notebook demonstrates various prompting techniques using zero-shot, few-shot, and chain-of-thought approaches. It also includes a medical diagnosis assistant example.

In [44]:
# Import required libraries
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
import re

# Load environment variables from .env file
load_dotenv()

# Azure OpenAI configuration
AOAI_CONFIG = {
    "model": os.getenv("AZURE_OPENAI_MODEL", "gpt4-32k"),
    "api_key": os.getenv("AZURE_OPENAI_API_KEY"),
    "base_url": os.getenv("AZURE_OPENAI_ENDPOINT"),
    "api_version": os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-01")
}

# Initialize Azure OpenAI client
client = AzureOpenAI(
    api_key=AOAI_CONFIG["api_key"],
    azure_endpoint=AOAI_CONFIG["base_url"],
    api_version=AOAI_CONFIG["api_version"]
)

def get_ai_response(prompt, system_message="You are a helpful assistant."):
    """
    Get a response from the AI model.
    """
    response = client.chat.completions.create(
        temperature=0.3,
        model=AOAI_CONFIG["model"],
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

def extract_answer(text):
    """
    Extract the answer from the AI's response enclosed in <ans> tags.
    """
    pattern = r'<ans>(.*?)</ans>'
    matches = re.findall(pattern, text, re.DOTALL)
    if matches:
        return matches[0].strip()
    return None

## Zero-Shot Sentiment Analysis

This example demonstrates zero-shot sentiment analysis, where the model classifies the sentiment of a movie review without any specific examples or training.

In [45]:
zero_shot_prompt = """
Classify the sentiment of the following movie review as positive or negative
"""

query = """ "I couldn't stand this movie. The plot was confusing, the acting was terrible, and I was bored throughout the entire film."  """
response = get_ai_response(query, zero_shot_prompt)
print(response)

The sentiment of this movie review is negative.


## Few-Shot Sentiment Analysis

This example shows few-shot sentiment analysis, where we provide the model with a few examples before asking it to classify a new review.

In [46]:
few_shot_prompt = """
Classify the sentiment of the following movie reviews as positive or negative.

Review: "This movie was absolutely terrible. I hated every minute of it."
Sentiment: Negative

Review: "I loved this film! The acting was superb and the plot kept me engaged throughout."
Sentiment: Positive

Review: "An instant classic! This movie will be remembered for years to come."
Sentiment: Positive
"""

query =  """ Review: "The special effects were impressive, but the story was confusing and hard to follow."""

response = get_ai_response(query, few_shot_prompt)

print(response)

Sentiment: Negative


## Chain-of-Thought Sentiment Analysis

This example demonstrates chain-of-thought sentiment analysis, where we guide the model through a step-by-step reasoning process.

In [47]:
cot_prompt = """
Analyze the sentiment of the following movie review step by step:

Review: "While the cinematography was breathtaking, the plot left much to be desired. The characters were poorly developed, and I found myself checking my watch multiple times. However, the soundtrack was quite good."

Step 1: Identify the main aspects mentioned in the review
- Cinematography
- Plot
- Character development
- Pacing
- Soundtrack

Step 2: Analyze the sentiment for each aspect
- Cinematography: Positive ("breathtaking")
- Plot: Negative ("left much to be desired")
- Character development: Negative ("poorly developed")
- Pacing: Negative ("checking my watch multiple times")
- Soundtrack: Positive ("quite good")

Step 3: Weigh the positive and negative aspects
There are more negative aspects (3) than positive ones (2), and the negative aspects seem to outweigh the positives in importance.

Step 4: Conclude the overall sentiment
Given that the core elements of a movie (plot, characters, and pacing) are criticized, while only technical aspects (cinematography and soundtrack) are praised, the overall sentiment leans negative.

Therefore, the overall sentiment of this review is negative.

Now, analyze the sentiment of this review using the same step-by-step approach:
"""
query =  """ "The movie started strong with an intriguing premise, but it quickly fell apart in the second act. The lead actor gave a stellar performance, yet the supporting cast seemed out of their depth. The special effects were top-notch, though they couldn't save the convoluted storyline."
"""
response = get_ai_response(query, cot_prompt)
print(response)

Step 1: Identify the main aspects mentioned in the review
- Initial premise
- Second act
- Lead actor's performance
- Supporting cast's performance
- Special effects
- Storyline

Step 2: Analyze the sentiment for each aspect
- Initial premise: Positive ("started strong with an intriguing premise")
- Second act: Negative ("quickly fell apart")
- Lead actor's performance: Positive ("stellar performance")
- Supporting cast's performance: Negative ("seemed out of their depth")
- Special effects: Positive ("top-notch")
- Storyline: Negative ("convoluted")

Step 3: Weigh the positive and negative aspects
There are an equal number of positive (3) and negative (3) aspects. However, the negative comments about the second act, supporting cast, and storyline could be seen as more significant than the positive remarks about the initial premise, lead actor's performance, and special effects.

Step 4: Conclude the overall sentiment
Given that the negative aspects relate to crucial elements of the mo

## Medical Diagnosis Usecase

This example showcases a medical diagnosis assistant using chain-of-thought reasoning to analyze patient symptoms and suggest potential diagnoses.

### Zero-Shot Medical Diagnosis

This example demonstrates a zero-shot approach to medical diagnosis, where we provide no examples and ask the model to suggest possible conditions based on symptoms.

In [48]:
medical_prompt = """
You are an AI medical assistant capable of preliminary symptom analysis. Given a patient's symptoms,
use your general medical knowledge to suggest possible conditions and recommend appropriate next steps.
"""


query = """
A 42-year-old woman reports experiencing frequent headaches, blurred vision, and occasional dizziness for the past three weeks. She mentions that the headaches are typically worse in the morning and improve throughout the day. She has no prior history of migraines or head injuries. The patient works long hours at a computer and has noticed increased stress at work recently. She also mentions that she has been waking up multiple times during the night to urinate, which is unusual for her.
"""

response = get_ai_response(query, medical_prompt)
print(response)



The symptoms you're describing could be due to a number of conditions. The headaches, blurred vision, and dizziness could be related to stress, eye strain from prolonged computer use, or high blood pressure. However, the frequent urination at night (known as nocturia) could suggest a urinary tract infection, diabetes, or a kidney disorder.

Given the range of symptoms and possible underlying conditions, it's important to seek medical attention. I would recommend you to:

1. Schedule an appointment with your primary care physician as soon as possible. They may perform a physical examination and order some tests, such as blood tests and urine tests, to rule out conditions like diabetes or a urinary tract infection.

2. Consider seeing an ophthalmologist or optometrist for an eye examination. Prolonged computer use can lead to a condition called computer vision syndrome, which can cause symptoms like headaches and blurred vision.

3. Try to manage stress levels. Stress can exacerbate many

### Chain-of-Thought (CoT) Medical Diagnosis

This example shows a chain-of-thought approach to medical diagnosis, where we guide the model through a step-by-step reasoning process.

In [49]:
medical_prompt = """
You are an AI assistant trained to help identify potential medical conditions based on patient symptoms. Use the following Chain-of-Thought process to analyze the patient's situation and suggest possible diagnoses:

Step 1: Gather and summarize key patient information
- List the patient's age, gender, and any relevant medical history
- Identify and list all symptoms mentioned by the patient

Step 2: Analyze the symptoms
- Group related symptoms together
- Note the duration and severity of each symptom
- Identify any potential triggering factors or patterns

Step 3: Consider common conditions
- List potential diagnoses that match the symptom profile
- Prioritize conditions based on symptom alignment and patient demographics

Step 4: Evaluate risk factors
- Consider the patient's age, gender, and medical history as potential risk factors
- Identify any lifestyle or environmental factors that may contribute to the condition

Step 5: Suggest potential diagnostic tests
- List relevant tests that could help confirm or rule out potential diagnoses
- Explain why each test would be helpful in this case

Step 6: Summarize findings
- Provide a concise summary of the most likely condition(s) based on the analysis
- Explain the reasoning behind your conclusions
- Mention any important differential diagnoses that should be considered

Step 7: Recommend next steps
- Suggest appropriate medical professionals to consult (e.g., primary care physician, specialist)
- Advise on any immediate actions the patient should take

Remember, this is not a definitive diagnosis, but a preliminary analysis to guide further medical investigation. Always advise the patient to consult with a qualified healthcare professional for an accurate diagnosis and treatment plan.


"""

query = """ A 42-year-old woman reports experiencing frequent headaches, blurred vision, and occasional dizziness for the past three weeks. She mentions that the headaches are typically worse in the morning and improve throughout the day. She has no prior history of migraines or head injuries. The patient works long hours at a computer and has noticed increased stress at work recently. She also mentions that she has been waking up multiple times during the night to urinate, which is unusual for her.
 """
response = get_ai_response(query, medical_prompt)
print(response)



Step 1: Gather and summarize key patient information
- Age: 42
- Gender: Female
- Relevant medical history: No prior history of migraines or head injuries
- Symptoms: Frequent headaches, blurred vision, occasional dizziness, nocturia (waking up to urinate at night), increased stress at work, long hours at a computer

Step 2: Analyze the symptoms
- Headaches, blurred vision, and dizziness are potentially related, suggesting a neurological or ocular issue.
- Headaches are worse in the morning and improve throughout the day.
- The patient has been experiencing these symptoms for the past three weeks.
- Increased stress at work and long hours at a computer could be contributing factors.
- Nocturia is a new symptom for the patient.

Step 3: Consider common conditions
- Potential diagnoses include tension headaches, high blood pressure, diabetes, or a neurological condition such as a brain tumor.

Step 4: Evaluate risk factors
- Age: Middle-aged, which increases risk for conditions like high

### 3-Shot Chain-of-Thought Medical Diagnosis

This example demonstrates a 3-shot chain-of-thought approach to medical diagnosis, where we provide three examples before asking the model to analyze a new case.


In [50]:
# Example: Chain-of-Thought
cot_prompt = """
You are an AI medical assistant trained to analyze patient symptoms and suggest potential diagnoses. Use the following examples to guide your thought process when presented with new patient cases:

Example 1:
Patient: A 28-year-old woman complains of a sore throat and fever for the past 2 days.

Thought process:
1. Key symptoms: sore throat, fever
2. Duration: 2 days
3. Patient characteristics: young adult female
4. Potential diagnoses:
   a) Viral pharyngitis
   b) Strep throat
   c) Common cold
5. Analysis:
   - Short duration suggests acute infection
   - Fever indicates possible bacterial infection
6. Prioritized diagnoses:
   1. Strep throat - matches symptoms and acuity
   2. Viral pharyngitis - common but less likely with fever
7. Recommendations:
   - Throat swab for rapid strep test
   - Symptomatic treatment (rest, hydration, pain relief)
   - If strep positive, consider antibiotics

Example 2:
Patient: A 45-year-old man reports chest pain, shortness of breath, and fatigue for the past week.

Thought process:
1. Key symptoms: chest pain, shortness of breath, fatigue
2. Duration: one week
3. Patient characteristics: middle-aged male
4. Potential diagnoses:
   a) Coronary artery disease (CAD)
   b) Pneumonia
   c) Anxiety or panic disorder
5. Analysis:
   - Chest pain + shortness of breath suggest possible cardiac or respiratory issues
   - Fatigue could be related to both cardiac and respiratory problems
   - Age and gender increase risk for CAD
6. Prioritized diagnoses:
   1. CAD - fits age group and symptom profile
   2. Pneumonia - possible but less likely without fever or cough
   3. Anxiety - consider as differential diagnosis
7. Recommendations:
   - Urgent evaluation by a physician
   - ECG and cardiac enzyme tests to rule out acute coronary syndrome
   - Chest X-ray to assess for pneumonia
   - Further history taking to determine nature of chest pain and any triggers

Example 3:
Patient: A 62-year-old woman with a history of hypertension presents with sudden onset of severe headache, confusion, and left-sided weakness.

Thought process:
1. Key symptoms: sudden severe headache, confusion, left-sided weakness
2. Duration: sudden onset (acute)
3. Patient characteristics: older adult female with hypertension history
4. Potential diagnoses:
   a) Stroke (ischemic or hemorrhagic)
   b) Transient Ischemic Attack (TIA)
   c) Complicated migraine
   d) Brain tumor
5. Analysis:
   - Sudden onset of neurological symptoms suggests vascular event
   - Left-sided weakness indicates right-sided brain involvement
   - Hypertension is a significant risk factor for stroke
   - Confusion could indicate large vessel involvement or increased intracranial pressure
6. Prioritized diagnoses:
   1. Acute stroke (high priority) - matches symptoms, risk factors, and acuity
   2. TIA - similar presentation but typically resolves within 24 hours
   3. Complicated migraine - less likely given age and presentation
   4. Brain tumor - possible but less likely given acute onset
7. Recommendations:
   - Immediate emergency medical services activation (stroke alert)
   - Urgent CT scan to differentiate between ischemic and hemorrhagic stroke
   - Neurological examination and NIH Stroke Scale assessment
   - If ischemic stroke confirmed and within time window, consider thrombolysis or mechanical thrombectomy
   - Blood pressure management and supportive care

"""


query = """
A 42-year-old woman reports experiencing frequent headaches, blurred vision, and occasional dizziness for the past three weeks. She mentions that the headaches are typically worse in the morning and improve throughout the day. She has no prior history of migraines or head injuries. The patient works long hours at a computer and has noticed increased stress at work recently. She also mentions that she has been waking up multiple times during the night to urinate, which is unusual for her.
"""

response = get_ai_response(query, medical_prompt)
print(response)

Step 1: Gather and summarize key patient information
- Age: 42 years
- Gender: Female
- Relevant medical history: No prior history of migraines or head injuries
- Symptoms: Frequent headaches, blurred vision, occasional dizziness, increased urination at night, and increased stress at work

Step 2: Analyze the symptoms
- Grouped symptoms: Headaches, blurred vision, and dizziness may be related. Increased urination at night could be a separate issue or related to the other symptoms.
- Duration and severity: Symptoms have been present for three weeks. Headaches are worse in the morning and improve throughout the day.
- Potential triggering factors or patterns: Long hours at a computer and increased stress at work. 

Step 3: Consider common conditions
- Potential diagnoses: Tension headaches, stress-related disorders, high blood pressure, diabetes, or a neurological condition like a brain tumor.

Step 4: Evaluate risk factors
- Age and gender: Middle-aged women are at increased risk for te

### Benefits of the Production-Ready Chain-of-Thought Approach

The production-ready Chain-of-Thought (CoT) approach offers significant advantages for both end-users and developers:

1. **For Customers: Clear, Concise Answers**
   - The `<ans>` tags encapsulate the final output, allowing systems to:
     - Present only the relevant conclusion to end-users
     - Provide clear, actionable information without overwhelming details
     - Deliver a consistent user experience across different queries

2. **For Developers: Transparent Reasoning Process**
   - The `<thinking>` tags preserve the AI's reasoning, enabling developers to:
     - Debug and improve the AI's logic by examining its step-by-step thought process
     - Audit the AI's decision-making for accuracy and bias
     - Refine prompts and model behavior based on detailed insights into the AI's reasoning
     - Implement robust error handling and quality control measures

This dual approach ensures that while customers receive straightforward, useful responses, developers retain full visibility into the AI's decision-making process.




In [51]:
medical_prompt = """
You are an AI medical assistant trained to analyze patient symptoms and suggest potential diagnoses. Use the following example to guide your thought process when presented with a new patient case. Enclose your reasoning within <thinking></thinking> tags and your final answer within <ans></ans> tags.

Example:
Patient: A 45-year-old man complains of chest pain, shortness of breath, and fatigue for the past week.

<thinking>
1. Identify key symptoms: chest pain, shortness of breath, fatigue
2. Consider duration: symptoms present for one week
3. Assess risk factors: middle-aged male
4. Generate potential diagnoses:
   a) Coronary artery disease (CAD)
   b) Pneumonia
   c) Anxiety or panic disorder
5. Analyze symptom patterns:
   - Chest pain + shortness of breath suggest possible cardiac or respiratory issues
   - Fatigue could be related to both cardiac and respiratory problems
6. Prioritize diagnoses:
   1. CAD - fits age group and symptom profile
   2. Pneumonia - possible but less likely without fever or cough
   3. Anxiety - consider as differential diagnosis
7. Recommend next steps:
   - Urgent evaluation by a physician
   - ECG and cardiac enzyme tests to rule out acute coronary syndrome
   - Chest X-ray to assess for pneumonia
   - Further history taking to determine nature of chest pain and any triggers
</thinking>

<ans>
Based on the symptoms and risk factors, the most likely diagnosis is Coronary Artery Disease (CAD). However, pneumonia and anxiety should also be considered. The patient requires urgent medical evaluation, including ECG, cardiac enzyme tests, and a chest X-ray. A detailed history of the chest pain characteristics is crucial for accurate diagnosis.
</ans>

Now, analyze the following patient case:
"""

query = """
A 42-year-old woman reports experiencing frequent headaches, blurred vision, and occasional dizziness for the past three weeks. She mentions that the headaches are typically worse in the morning and improve throughout the day. She has no prior history of migraines or head injuries. The patient works long hours at a computer and has noticed increased stress at work recently. She also mentions that she has been waking up multiple times during the night to urinate, which is unusual for her.
"""

response = get_ai_response(query, medical_prompt)
print(response)



<thinking>
1. Identify key symptoms: frequent headaches, blurred vision, occasional dizziness, nocturia (waking up to urinate at night)
2. Consider duration: symptoms present for three weeks
3. Assess risk factors: long hours at a computer, increased stress at work
4. Generate potential diagnoses:
   a) Brain tumor
   b) Diabetes
   c) Glaucoma
   d) Stress or tension headaches
5. Analyze symptom patterns:
   - Headaches that are worse in the morning and improve throughout the day, along with blurred vision and dizziness, could suggest increased intracranial pressure, which is seen in conditions like brain tumors
   - Nocturia can be a symptom of diabetes, but there are no other typical symptoms like excessive thirst or weight loss
   - Blurred vision can be a symptom of glaucoma, but this condition doesn't typically cause headaches or dizziness
   - Long hours at a computer and increased stress could cause tension headaches, but these wouldn't explain the nocturia or blurred vision
6.

Filter the answer for the user

In [52]:
answer = extract_answer(response)
if answer:
    print(answer)

Based on the symptoms and risk factors, a brain tumor could be a possible diagnosis, but further testing is required to confirm. Diabetes and glaucoma should also be considered, as well as stress or tension headaches. The patient should seek urgent medical evaluation, including blood tests, an eye examination, and possibly an MRI or CT scan of the brain. A detailed history of the headaches and any potential triggers is also crucial for accurate diagnosis.


## Conclusion

1. **Zero-Shot Prompting**:
   - Simplest approach, relying on the model's pre-trained knowledge
   - Effective for straightforward tasks but limited for complex medical diagnoses

2. **Few-Shot Prompting**:
   - Introduces examples to guide the AI, improving relevance and accuracy
   - Helps the model understand the desired output format and context

3. **Chain-of-Thought (CoT) Prompting**:
   - Encourages step-by-step reasoning, enhancing the AI's analytical capabilities
   - Provides transparency into the AI's decision-making process

4. **3-Shot CoT**:
   - Refines the CoT approach with multiple examples
   - Improves consistency and depth of analysis across various medical scenarios

5. **Production-Ready CoT**:
   - Structures prompts for real-world applications
   - Separates thinking process (`<thinking>` tags) from final answers (`<ans>` tags)
   - Balances detailed analysis for developers with concise outputs for end-users
